WORD2VEC and ELMO


-->Install flair and restarting runtime

In [2]:
!pip install allennlp==0.9.0
!pip install --user flair

  Using cached conllu-1.3.1-py2.py3-none-any.whl (9.3 kB)
  Attempting uninstall: conllu
    Found existing installation: conllu 4.4.1
    Uninstalling conllu-4.4.1:
      Successfully uninstalled conllu-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flair 0.11.1 requires conllu>=4.0, but you have conllu 1.3.1 which is incompatible.
  Using cached conllu-4.4.1-py2.py3-none-any.whl (15 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 0.9.0 requires conllu==1.3.1, but you have conllu 4.4.1 which is incompatible.


1)Mounting Gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2)Loading Dataset

In [3]:
from zipfile import ZipFile

In [3]:
with ZipFile('drive/MyDrive/Dataset/quora-question-pairs.zip','r') as ZipObj:
  ZipObj.extractall('drive/MyDrive/Dataset')

In [4]:
with ZipFile('drive/MyDrive/Dataset/sample_submission.csv.zip','r') as ZipObj:
  ZipObj.extractall('drive/MyDrive/Dataset/Train_Test')

with ZipFile('drive/MyDrive/Dataset/train.csv.zip','r') as ZipObj:
  ZipObj.extractall('drive/MyDrive/Dataset/Train_Test')

with ZipFile('drive/MyDrive/Dataset/test.csv.zip','r') as ZipObj:
  ZipObj.extractall('drive/MyDrive/Dataset/Train_Test')

In [2]:
import pandas as pd

In [23]:
train=pd.read_csv('drive/MyDrive/Dataset/Train_Test/train.csv')
test=pd.read_csv('drive/MyDrive/Dataset/Train_Test/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
train.shape,test.shape

((404290, 6), (3563475, 3))

In [5]:
train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [6]:
test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [4]:
train=train[0:1000]
test=test[0:1000]
train.shape,test.shape

((1000, 6), (1000, 3))

3) Loading Frequencies

In [5]:
import csv

def load_tsv(f):
    frequencies = {}
    with open(f) as tsv:
        tsv_reader = csv.reader(tsv, delimiter="\t")
        for row in tsv_reader:
            frequencies[row[0]] = int(row[1])

    return frequencies


def load_frequencies(path):
    return load_tsv(path)


def load_doc_frequencies(path):
    doc_frequencies = load_tsv(path)
    doc_frequencies["NUM_DOCS"] = 1288431
    return doc_frequencies

In [6]:
frequency = load_frequencies("drive/MyDrive/Code_Frequencies_Embeddings/frequencies.tsv")
doc_frequency = load_doc_frequencies("drive/MyDrive/Code_Frequencies_Embeddings/doc_frequencies.tsv")

4) Loading Embeddings

In [7]:
import gensim

def load_word2vec(path):
    word2vec = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)
    return word2vec

In [8]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('drive/MyDrive/Code_Frequencies_Embeddings/GoogleNews-vectors-negative300.bin.gz').extractall('drive/MyDrive/Code_Frequencies_Embeddings')

In [9]:
word2vec = load_word2vec("drive/MyDrive/Code_Frequencies_Embeddings/GoogleNews-vectors-negative300.bin")

In [10]:
from flair.embeddings import ELMoEmbeddings
elmo = ELMoEmbeddings('original')

5) Similarity Measure- Vector Average

In [11]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


In [12]:
import math
from collections import Counter

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def run_avg_benchmark(sentences1, sentences2, model=None, use_stoplist=False, doc_freqs=None):
    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            sims.append(0)
            continue

        tokfreqs1 = Counter(tokens1)
        tokfreqs2 = Counter(tokens2)

        weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs1] if doc_freqs else None
        weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs2] if doc_freqs else None

        embedding1 = np.average([model[token] for token in tokfreqs1], axis=0, weights=weights1).reshape(1, -1)
        embedding2 = np.average([model[token] for token in tokfreqs2], axis=0, weights=weights2).reshape(1, -1)

        sim = cosine_similarity(embedding1, embedding2)[0][0]
        sims.append(sim)

    return sims


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
STOP = set(nltk.corpus.stopwords.words("english"))


class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        #normalized_sentence = sentence.replace("'", "'").replace("'", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
def run_experiment(df, benchmark):
    sentences1 = [SentenceModel(s) for s in df['question1']]
    sentences2 = [SentenceModel(s) for s in df['question2']]

    sims = benchmark[1](sentences1, sentences2)

    return sims, benchmark[0]


In [15]:
import math
from collections import Counter

import numpy as np
from flair.data import Sentence
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm


def run_context_avg_benchmark(sentences1, sentences2, model=None, use_stoplist=False, doc_freqs=None):
    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    sims = []
    for (sent1, sent2) in tqdm(zip(sentences1, sentences2), total=len(sentences1)):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        flair_tokens1 = sent1.tokens
        flair_tokens2 = sent2.tokens

        flair_sent1 = Sentence(" ".join(flair_tokens1))
        flair_sent2 = Sentence(" ".join(flair_tokens2))

        model.embed(flair_sent1)
        model.embed(flair_sent2)

        embeddings_map1 = {}
        embeddings_map2 = {}

        for token in flair_sent1:
            embeddings_map1[token.text] = np.array(token.embedding.data.tolist())

        for token in flair_sent2:
            embeddings_map2[token.text] = np.array(token.embedding.data.tolist())

        if len(tokens1) == 0 or len(tokens2) == 0:
            sims.append(0)
            continue

        tokfreqs1 = Counter(tokens1)
        tokfreqs2 = Counter(tokens2)

        weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs1 if token in embeddings_map1] if doc_freqs else None
        weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs2 if token in embeddings_map2] if doc_freqs else None

        embedding1 = np.average([embeddings_map1[token] for token in tokfreqs1 if token in embeddings_map1], axis=0, weights=weights1).reshape(1, -1)
        embedding2 = np.average([embeddings_map2[token] for token in tokfreqs2 if token in embeddings_map2], axis=0, weights=weights2).reshape(1, -1)

        sim = cosine_similarity(embedding1, embedding2)[0][0]
        sims.append(sim)

    return sims


In [16]:
import functools as ft
import pandas as pd
import matplotlib.pyplot as plt

benchmarks = [("AVG-W2V", ft.partial(run_avg_benchmark, model=word2vec, use_stoplist=False)),
              ("AVG-ELMO", ft.partial(run_context_avg_benchmark, model=elmo, use_stoplist=False)),
              ("AVG-W2V-STOP", ft.partial(run_avg_benchmark, model=word2vec, use_stoplist=True)),
              ("AVG-ELMO-STOP", ft.partial(run_context_avg_benchmark, model=elmo, use_stoplist=True)),
              ("AVG-W2V-TFIDF", ft.partial(run_avg_benchmark, model=word2vec, use_stoplist=False, doc_freqs=doc_frequency)),
              ("AVG-ELMO-TFIDF", ft.partial(run_context_avg_benchmark, model=elmo, use_stoplist=False, doc_freqs=doc_frequency)),
              ("AVG-W2V-TFIDF-STOP",ft.partial(run_avg_benchmark, model=word2vec, use_stoplist=True, doc_freqs=doc_frequency)),
              ("AVG-ELMO-TFIDF-STOP",ft.partial(run_context_avg_benchmark, model=elmo, use_stoplist=True, doc_freqs=doc_frequency))]

In [133]:
training=train.copy()

In [116]:
del training['id']
del training['qid1']
del training['qid2']

In [117]:
import time

In [ ]:
import scipy
from sklearn.metrics import mean_squared_error
from math import sqrt

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(25,20))
row = 0
column = 0
training = normalize(training, ["is_duplicate"])


start_time = time.time()

for i in range(0, 8):
    sims, topic = run_experiment(training, benchmarks[i])
    pearson_correlation = scipy.stats.pearsonr(sims, training['is_duplicate'])[0]
    spearman_correlation = scipy.stats.spearmanr(sims, training['is_duplicate'])[0]
    rmse = sqrt(mean_squared_error(sims, training['is_duplicate']))
    textstr = 'RMSE=%.3f\n$Pearson Correlation=%.3f$\n$Spearman Correlation=%.3f$'%(rmse, pearson_correlation, spearman_correlation)
    training['predicted_sim'] = pd.Series(sims).values
    training = training.sort_values('is_duplicate')
    id = list(range(0, len(training.index)))
    training['id'] = pd.Series(id).values
    
    if(i < 4):
        row = 0
        column = i
    if(i >= 4 and i < 8):
        row = 1
        column = i-4

    print("--- %s seconds ---" % (time.time() - start_time))
        
    training.plot(kind='scatter', x='id', y='sim',color='DarkBlue', label='Similarity', title = topic, ax=axes[row, column]);
    training.plot(kind='scatter', x='id', y='predicted_sim', color='DarkGreen', label='Predicted Similarity', ax=axes[row, column]);
    axes[row, column].text(1500, 0, textstr, fontsize=12)
    print (topic)
    print (textstr)

6) Similarity Measure- Smooth Inverse Frequency

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    print(X)
    print(type(X))
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_sif_benchmark(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=0.001):
    total_freq = sum(freqs.values())

    embeddings = []

    # SIF requires us to first collect all sentence embeddings and then perform
    # common component analysis.
    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
        weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

        embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)

        embeddings.append(embedding1)
        embeddings.append(embedding2)

    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
            for idx in range(int(len(embeddings) / 2))]

    return sims


In [ ]:
import numpy as np
from flair.data import Sentence
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_context_sif_benchmark(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=0.001):
    total_freq = sum(freqs.values())

    embeddings = []

    # SIF requires us to first collect all sentence embeddings and then perform
    # common component analysis.
    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        flair_tokens1 = sent1.tokens
        flair_tokens2 = sent2.tokens

        flair_sent1 = Sentence(" ".join(flair_tokens1))
        flair_sent2 = Sentence(" ".join(flair_tokens2))

        model.embed(flair_sent1)
        model.embed(flair_sent2)

        embeddings_map1 = {}
        embeddings_map2 = {}

        for token in flair_sent1:
            embeddings_map1[token.text] = np.array(token.embedding.data.tolist())

        for token in flair_sent2:
            embeddings_map2[token.text] = np.array(token.embedding.data.tolist())

        tokens1 = [token for token in tokens1]
        tokens2 = [token for token in tokens2]

        weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
        weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

        embedding1 = np.average([embeddings_map1[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([embeddings_map2[token] for token in tokens2], axis=0, weights=weights2)

        embeddings.append(embedding1)
        embeddings.append(embedding2)

    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
            for idx in range(int(len(embeddings) / 2))]

    return sims


In [ ]:
import functools as ft

benchmarks = [("SIF-W2V", ft.partial(run_sif_benchmark, freqs=frequency, model=word2vec, use_stoplist=False)),
              ("SIF-ELMO", ft.partial(run_context_sif_benchmark, freqs=frequency, model=elmo, use_stoplist=False))]

In [ ]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
STOP = set(nltk.corpus.stopwords.words("english"))


class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        #normalized_sentence = sentence.replace("'", "'").replace("'", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]

In [ ]:
def run_experiment(df, benchmark):
    sentences1 = [SentenceModel(s) for s in df['question1']]
    sentences2 = [SentenceModel(s) for s in df['question2']]

    sims = benchmark[1](sentences1, sentences2)

    return sims, benchmark[0]


In [ ]:
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,10))
row = 0
column = 0
training = normalize(training, ["is_duplicate"])
for i in range(0, 2):
    sims, topic = run_experiment(training, benchmarks[i])
    training['predicted_sim'] = pd.Series(sims).values
    training = normalize(training, ["predicted_sim"])
    pearson_correlation = scipy.stats.pearsonr(sims, training['sim'])[0]
    spearman_correlation = scipy.stats.spearmanr(sims, training['sim'])[0]
    rmse = sqrt(mean_squared_error(training['predicted_sim'], training['sim']))
    textstr = 'rmse=%.3f\n$Pearson Correlation=%.3f$\n$Spearman Correlation=%.3f$'%(rmse, pearson_correlation, spearman_correlation)    
    training = training.sort_values('sim')
    id = list(range(0, len(training.index)))
    training['id'] = pd.Series(id).values
    
    if(i < 2):
        column = i
        
    training.plot(kind='scatter', x='id', y='sim',color='DarkBlue', label='Similarity', title = topic, ax=axes[column]);
    training.plot(kind='scatter', x='id', y='predicted_sim', color='DarkGreen', label='Predicted Similarity', ax=axes[column]);
    axes[column].text(1500, 0.05, textstr, fontsize=12)
    print (topic)
    print (textstr)